In [1]:
# Check the current version
!pip show google-generativeai

# Upgrade to the latest version
!pip install --upgrade google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 202.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 492.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 393.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 510.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 207.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.4
    Uninstalling protobuf-4.25.4:
      Successfully uninstalled protobuf-4.25.4
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.66.0
    Uninstalling grpcio-1.66.0:
      Successfully uninstalled grpcio-1.66.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.8.0 req

In [2]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.1 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import google.generativeai as genai
import re
import PyPDF2

In [10]:
# Configure Google Generative AI with your API key

genai.configure(api_key="*******")

model = genai.GenerativeModel('gemini-pro')

# Function to extract text from a PDF document
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        extracted_text = ""
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                extracted_text += text
        return extracted_text

# Function to parse user stories from extracted text
def parse_user_stories(text):
    user_stories = {}
    matches = re.findall(r'(User Story \d+:)(.*?)(?=User Story \d+:|$)', text, re.DOTALL)
    for match in matches:
        story_id, story_text = match[0].strip(), match[1].strip()
        user_stories[story_id] = story_text
    return user_stories

# Function to generate either an effort estimate or Node.js implementation guide
def generate_response(user_story, mode="estimate"):
    if mode == "estimate":
        prompt = f"Estimate the development effort in hours required for the following user story:\n\n{user_story}"
    elif mode == "guide":
        prompt = f"""
        Provide a detailed guide on how to implement the following user story as a backend service using Node.js:
        User Story: '{user_story}'

        Include specific instructions on:
        1. Necessary route endpoints and HTTP methods.
        2. Sample code snippets for Express routes and middleware.
        3. Recommended data schema using MongoDB and Mongoose.
        4. Steps for user authentication or authorization if required.
        5. Any other relevant backend logic.
        """
    else:
        raise ValueError("Mode should be either 'estimate' or 'guide'")

    response = model.generate_content(prompt)
    return response.text.strip()

# Function to clean up the extracted text for easier processing
def clean_text(text):
    text = re.sub(r'\uf095', '', text)  # Remove bullet icons
    text = re.sub(r'-\s+', '', text)    # Remove unnecessary line breaks
    text = re.sub(r'\s+', ' ', text)    # Replace multiple spaces/newlines
    text = re.sub(r'\s([?.!,"])', r'\1', text)  # Remove extra spaces around punctuation
    return text.strip()

# Further cleaning for non-functional and extraneous sections
def further_clean_text(text):
    text = re.sub(r'\b(\w+)\s*-\s*(\w+)\b', r'\1\2', text)
    text = re.sub(r'3\.\d+.*Requirements.*', '', text)
    text = re.sub(r'4\.\d+.*Requirements.*', '', text)
    text = re.sub(r'5\.\d+.*Requirements.*', '', text)
    return text.strip()

# Main function to analyze the SRS document
def analyze_srs(file_path, mode="estimate"):
    # Step 1: Extract text from PDF
    text = extract_text_from_pdf(file_path)

    # Step 2: Parse user stories
    user_stories = parse_user_stories(text)

    # Clean and further clean user story text
    cleaned_user_stories = {story_id: clean_text(story_text) for story_id, story_text in user_stories.items()}
    final_user_stories = {story_id: further_clean_text(story_text) for story_id, story_text in cleaned_user_stories.items()}

    # Print the final cleaned user stories
    for story_id, story_text in final_user_stories.items():
        print(f"{story_id}: {story_text}\n")

    # Step 3: Generate response based on the specified mode
    results = {}
    for story_id, story_text in final_user_stories.items():
        response = generate_response(story_text, mode=mode)
        results[story_id] = response

    # Display the results
    for story_id, response in results.items():
        if mode == "estimate":
            print(f"{story_id} Estimated Effort:\n{response}\n")
        elif mode == "guide":
            print(f"{story_id} Implementation Guide:\n{response}\n")

# Run the analysis on the SRS document
file_path = "srs.pdf"

# Set mode to "estimate" for effort estimation or "guide" for implementation guide
analyze_srs(file_path, mode="estimate")  # or mode="guide" depending on your needs


User Story 1:: User Registration and Profile Management Description : As a user, I want to create an account and manage my profile so I can personalize my travel experience. Acceptance Criteria : Users can register using an email or social media account. Users can update their personal information, including preferences for sustainable travel.

User Story 2:: Sustai nable Route Planning Description : As a user, I want to plan my route based on the lowest environmental impact so that I can minimize my carbon footprint. Acceptance Criteria : Users can input their starting location and destination. The app provides route o ptions with CO2 estimates and categorizes routes by eco friendliness. Users can view estimated carbon savings when choosing a sustainable route.

User Story 3:: EcoFriendly Accommodation Finder Description : As a user, I want to find accommodations with ecofriendly certifications so that I can support sustainable businesses. Acceptance Criteria : The app provides a list

In [11]:
# Configure Google Generative AI with your API key

genai.configure(api_key="*********")

model = genai.GenerativeModel('gemini-pro')

# Function to extract text from a PDF document
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        extracted_text = ""
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                extracted_text += text
        return extracted_text

# Function to parse user stories from extracted text
def parse_user_stories(text):
    user_stories = {}
    matches = re.findall(r'(User Story \d+:)(.*?)(?=User Story \d+:|$)', text, re.DOTALL)
    for match in matches:
        story_id, story_text = match[0].strip(), match[1].strip()
        user_stories[story_id] = story_text
    return user_stories

# Function to generate either an effort estimate or Node.js implementation guide
def generate_response(user_story, mode="estimate"):
    if mode == "estimate":
        prompt = f"Estimate the development effort in hours required for the following user story:\n\n{user_story}"
    elif mode == "guide":
        prompt = f"""
        Provide a detailed guide on how to implement the following user story as a backend service using Node.js:
        User Story: '{user_story}'

        Include specific instructions on:
        1. Necessary route endpoints and HTTP methods.
        2. Sample code snippets for Express routes and middleware.
        3. Recommended data schema using MongoDB and Mongoose.
        4. Steps for user authentication or authorization if required.
        5. Any other relevant backend logic.
        """
    else:
        raise ValueError("Mode should be either 'estimate' or 'guide'")

    response = model.generate_content(prompt)
    return response.text.strip()

# Function to clean up the extracted text for easier processing
def clean_text(text):
    text = re.sub(r'\uf095', '', text)  # Remove bullet icons
    text = re.sub(r'-\s+', '', text)    # Remove unnecessary line breaks
    text = re.sub(r'\s+', ' ', text)    # Replace multiple spaces/newlines
    text = re.sub(r'\s([?.!,"])', r'\1', text)  # Remove extra spaces around punctuation
    return text.strip()

# Further cleaning for non-functional and extraneous sections
def further_clean_text(text):
    text = re.sub(r'\b(\w+)\s*-\s*(\w+)\b', r'\1\2', text)
    text = re.sub(r'3\.\d+.*Requirements.*', '', text)
    text = re.sub(r'4\.\d+.*Requirements.*', '', text)
    text = re.sub(r'5\.\d+.*Requirements.*', '', text)
    return text.strip()

# Main function to analyze the SRS document
def analyze_srs(file_path, mode="estimate"):
    # Step 1: Extract text from PDF
    text = extract_text_from_pdf(file_path)

    # Step 2: Parse user stories
    user_stories = parse_user_stories(text)

    # Clean and further clean user story text
    cleaned_user_stories = {story_id: clean_text(story_text) for story_id, story_text in user_stories.items()}
    final_user_stories = {story_id: further_clean_text(story_text) for story_id, story_text in cleaned_user_stories.items()}

    # Print the final cleaned user stories
    for story_id, story_text in final_user_stories.items():
        print(f"{story_id}: {story_text}\n")

    # Step 3: Generate response based on the specified mode
    results = {}
    for story_id, story_text in final_user_stories.items():
        response = generate_response(story_text, mode=mode)
        results[story_id] = response

    # Display the results
    for story_id, response in results.items():
        if mode == "estimate":
            print(f"{story_id} Estimated Effort:\n{response}\n")
        elif mode == "guide":
            print(f"{story_id} Implementation Guide:\n{response}\n")

# Run the analysis on the SRS document
file_path = "srs.pdf"

# Set mode to "estimate" for effort estimation or "guide" for implementation guide
analyze_srs(file_path, mode="guide")  # or mode="guide" depending on your needs


User Story 1:: User Registration and Profile Management Description : As a user, I want to create an account and manage my profile so I can personalize my travel experience. Acceptance Criteria : Users can register using an email or social media account. Users can update their personal information, including preferences for sustainable travel.

User Story 2:: Sustai nable Route Planning Description : As a user, I want to plan my route based on the lowest environmental impact so that I can minimize my carbon footprint. Acceptance Criteria : Users can input their starting location and destination. The app provides route o ptions with CO2 estimates and categorizes routes by eco friendliness. Users can view estimated carbon savings when choosing a sustainable route.

User Story 3:: EcoFriendly Accommodation Finder Description : As a user, I want to find accommodations with ecofriendly certifications so that I can support sustainable businesses. Acceptance Criteria : The app provides a list